In [ ]:
## Day 5 class session
Embeddings

from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Load a language model
model = AutoModel.from_pretrained('bert-base-uncased')

##microsoft/deberta-v3-xsmall

# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]

output.shape

for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

output

### Scaled Dot Product Attention

from transformers import BertModel

model = BertModel.from_pretrained('bert-base-uncased')

len(model.encoder.layer)

model.encoder.layer[0]

model.encoder.layer[0].attention

## Multi Headed Attention

from transformers import BertConfig
config = BertConfig()
config

!pip install bertviz

from transformers import BertModel, BertTokenizer
from bertviz import head_view
import torch
import pandas as pd

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

text = "My friend told me about this series and I loved it so far! She was Right"

token = tokenizer.encode(text)

input = torch.tensor(token).unsqueeze(0)

input

attention = model(input, output_attentions=True)[2] #Extract Attention Weights from BERT model

attention

#average attention in all heads
final_attention = attention[-1].mean(1)[0]
final_attention.size()

import pandas as pd

attention_df = pd.DataFrame(final_attention.detach()).applymap(float).round(3)

attention_df.columns = tokenizer.convert_ids_to_tokens(token)
attention_df.index = tokenizer.convert_ids_to_tokens(token)

attention_df

tokens_as_list = tokenizer.convert_ids_to_tokens(input[0])
head_view(attention, tokens_as_list)

#2 - Previous & 6 - Pronouns

#previous tokens
head_view(attention,tokenizer.convert_ids_to_tokens(input[0]),layer=2, heads=[0])

#object - verb
head_view(attention,tokenizer.convert_ids_to_tokens(input[0]),layer=7, heads=[9])

## Day 4 afternoon session

!pip install bertviz

from transformers import pipeline, set_seed, AutoTokenizer, AutoModelForCausalLM, AutoModel
from torch import tensor, numel
from bertviz import model_view, head_view
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

set_seed(42)

# pipeline object in transformers provides easy access to transformer usage
MODEL = 'gpt2'

generator = pipeline('text-generation', model=MODEL)

# finish the sentence
generator("Hello, I'm a language model and I", max_length=30, num_return_sequences=3)

# load up a tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

'Sinan' in tokenizer.get_vocab()

tokenizer.encode('Sinan loves a beautiful day')

# encode a string and then convert the ids back into tokens. Note the Ġ character denoting a space before the token
tokenizer.convert_ids_to_tokens(tokenizer.encode('Sinan loves a beautiful day'))

tokenizer.encode('Sinan loves a beautiful day')  # ids

encoded = tokenizer.encode('Sinan loves a beautiful day', return_tensors='pt')  # as a pytorch tensor

encoded

# load up a tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForCausalLM.from_pretrained(MODEL)

model

encoded

model.transformer.wte(encoded)

model.transformer.wte(encoded).shape  # 1 item in batch x 6 tokens x token dimension

model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6)).shape  # manually create position vectors

# create GPT input
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6))

initial_input.shape

initial_input = model.transformer.drop(initial_input)  # run our input through the model's initual dropout later
initial_input

model.lm_head

for module in model.transformer.h:  # run the initial_input through every decoder in the stack
    initial_input = module(initial_input)[0]

initial_input = model.transformer.ln_f(initial_input)  # and then the final layer norm

initial_input

# same as just running through the model
(initial_input == model(encoded, output_hidden_states=True).hidden_states[-1]).all()

model(encoded).logits.shape

total_params = 0
for param in model.parameters():
    total_params += numel(param)

print(f'Number of params: {total_params:,}')

## Masked multi-headed attention**

import torch
import pandas as pd

phrase = 'My friend was right about this class. It is so fun!'
encoded_phrase = tokenizer(phrase, return_tensors='pt')

response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)

len(response.attentions)

encoded_phrase


response.attentions[-1].shape  # represtnations from the final decoder

encoded_phrase['input_ids'].shape

tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])

tokens

# Layer index 9, head 0. Check out the almost 60% attention the token it is giving to the token class
arr = response.attentions[9][0][0]

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)

attention_df.columns = tokens
attention_df.index = tokens

attention_df

head_view(response.attentions, tokens)

tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
model_view(response.attentions, tokens)

response.hidden_states[-1].shape

response.logits

response.logits.shape

# look at the top next token in the auto-regressive language modelling task
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
    columns=['Sequence up until', 'Next token with highest probability']
)

generator('My friend was right', max_length=12, num_return_sequences=5)

generator(phrase, max_length=20, num_return_sequences=1, do_sample=False)  # greedy search

generator(phrase, max_length=20, num_return_sequences=1, do_sample=True)  # greedy search with sampling

from transformers import AutoTokenizer

# Use the id of the model you want to use
# GPT-2 "openai-community/gpt2"
# Qwen "Qwen/Qwen2-0.5B"
# SmolLM "HuggingFaceTB/SmolLM-135M"

prompt = "It was a dark and stormy"
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
input_ids = tokenizer(prompt).input_ids
input_ids

for t in input_ids:
    print(t, "\t:", tokenizer.decode(t))

# We tokenize again but specifying the tokenizer that we want it to
# return a PyTorch tensor, which is what the model expects,
# rather than a list of integers
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model(input_ids)
outputs.logits.shape  # An output for each input token

final_logits = model(input_ids).logits[0, -1]  # The last set of logits
final_logits.argmax()  # The position of the maximum

tokenizer.decode(final_logits.argmax())

import torch

top10_logits = torch.topk(final_logits, 10)
for index in top10_logits.indices:
    print(tokenizer.decode(index))

top10 = torch.topk(final_logits.softmax(dim=0), 10)
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index):<10} {value.item():.2%}")

output_ids = model.generate(input_ids, max_new_tokens=20)
decoded_text = tokenizer.decode(output_ids[0])

print("Input IDs", input_ids[0])
print("Output IDs", output_ids)
print(f"Generated text: {decoded_text}")

beam_output = model.generate(
    input_ids,
    num_beams=5,
    max_new_tokens=30,
)

print(tokenizer.decode(beam_output[0]))

beam_output = model.generate(
    input_ids,
    num_beams=5,
    repetition_penalty=2.0,
    max_new_tokens=38,
)

print(tokenizer.decode(beam_output[0]))

## Do Sample

from transformers import set_seed

# Setting the seed ensures we get the same results every time we run this code
set_seed(70)

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=34,
    top_k=0,  # We'll come back to this parameter
)

print(tokenizer.decode(sampling_output[0]))

## Temperature

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.4,
    max_new_tokens=40,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.001,
    max_new_tokens=40,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=3.0,
    max_new_tokens=40,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

## Top-K

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=40,
    top_k=5,
)

print(tokenizer.decode(sampling_output[0]))

## Top - P

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=40,
    top_p=0.94,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

## Day 4 assessment

from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                         Trainer, TrainingArguments

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # load up a standard gpt2 model

tokenizer.pad_token = tokenizer.eos_token
# set our pad token to be the eos token. This lets gpt know how to fill space

# load up our data into a dataset
pds_data = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/Hands-On-Large-Language-Models.txt',  # Principles of Data Science - Sinan Ozdemir
    block_size=64  # length of each chunk of text to use as a datapoint
)

pds_data[0], pds_data[0].shape  # inspect the first point

print(tokenizer.decode(pds_data[0]))

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
    # MLM is Masked Language Modelling (for BERT + auto-encoding tasks)
)

# example of how collator pads data dynamically
collator_example = data_collator([tokenizer('I am an input'), tokenizer('So am I')])

collator_example

collator_example.input_ids  # 50256 is our pad token id

tokenizer.pad_token_id

collator_example.attention_mask  # Note the 0 in the attention mask where we have a pad token

collator_example.labels  # note the -100 to ignore loss calculation for the padded token
# Labels are shifted inside the GPT model so we don't need to worry about that

model = GPT2LMHeadModel.from_pretrained('gpt2')  # load up a GPT2 model

pretrained_generator = pipeline(  # create a generator with built in params
    'text-generation', model=model, tokenizer='gpt2',
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

print('----------')
for generated_sequence in pretrained_generator('what is agents', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

training_args = TrainingArguments(
    output_dir="./gpt2_pds", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=pds_data.examples[:int(len(pds_data.examples)*.8)],
    eval_dataset=pds_data.examples[int(len(pds_data.examples)*.8):]
)

trainer.evaluate()

trainer.train()

trainer.evaluate()  # loss decrease is slowing down so we are hitting our limit

trainer.save_model()

loaded_model = GPT2LMHeadModel.from_pretrained('./gpt2_pds')

finetuned_generator = pipeline(
    'text-generation', model=loaded_model, tokenizer=tokenizer,
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

# examples are now sustainably about data
print('----------')
for generated_sequence in finetuned_generator('what is agents', num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')